# Telegram bot with zAI

## Introduction

In this tutorial we will learn how to implement a simple telegram bot to process our photos using Python Telegram Bot and zAI. 

Telegram is a messaging app focused in the speed and security, which can be used in multiple devices such as smartphone, desktop, tablet, etc. at the same time. It also has the functionality of create bots to acomplish all kind of tasks. In this tutorial we will use <a href=https://github.com/python-telegram-bot/python-telegram-bot>Python Telegram Bot</a>, a library provides a Python interface for <a href=https://core.telegram.org/bots/api>Telegram Bot Api</a>. It also provides high-level classes to make the development of bots easy and straightforward. You can install or upgrade python-telegram-bot with:

## First steps in Python Telegram Bot

The first step is create a new Telegram Bot using **BotFather**. Simply introdocing _/newbot_ and following the steps the bot creates a new one that you can manage from it. You can access it from the API using its token. 

In [1]:
TOKEN = '<token>'

We create an updater and a dispatcher to simplify access:

In [2]:
from telegram.ext import Updater

In [3]:
updater  = Updater(token=TOKEN)
dispatcher = updater.dispatcher

The firts command we will create is */start*. It will works as welcome message and to get familiar with the sintaxis. We define a function to be called when the bot receive the message /start that replies _'Welcome to zAIbot!'_.

In [4]:
def cmd_start(bot, update):
    bot.send_message(chat_id = update.message.chat_id, text='Welcome to zAIbot')

Then, we create a command handler and add it to the dispatcher. 

In [5]:
from telegram.ext import CommandHandler

In [6]:
start_handler = CommandHandler('start', cmd_start)
dispatcher.add_handler(start_handler)

Finally, we start the bot with:

In [7]:
updater.start_polling()

To stop it, use:

In [8]:
updater.stop()

## zImage in Telegram

In this section, we will show how the class zImage of the API zAI could be use in our telegram bot. In this tutorial we are going to use Google as backend. We assume that the user has already configured properly the backend API's keys.

So, let's begin!

First, we will make that our bot shows us a menu with all the options that we have when we send it a photo. 

In [9]:
from telegram import ReplyKeyboardMarkup

In [10]:
def cmd_photo(bot, update):
    # Get id of the photo with the best quality
    photo_id = update.message.photo[-1].file_id
    # Get photo and save it as "newPhoto.png"
    newPhoto = bot.get_file(photo_id)
    newPhoto.download('newPhoto.png')
    # Create a custom reply keyboard
    reply_keyboard = [['/OCR','/findFaces', '/labels']]
    reply_markup = ReplyKeyboardMarkup(reply_keyboard)
    bot.send_message(chat_id = update.message.chat_id, text='What to do with the image?', reply_markup=reply_markup)

Instead of a CommandHandler, in this case we use a MessageHandler with a photo filter. In this case, the function cmd_phote will be called any time the bot receives a photo.

In [11]:
from telegram.ext import MessageHandler, Filters

In [12]:
photo_handler = MessageHandler(Filters.photo, cmd_photo)
dispatcher.add_handler(photo_handler)

One of the these options are OCR. This reads the text contained in the photo and send us a message with it. To do it, we define a function cmd_OCR that creates a new zImage object with the photo previously downloaded and applies optical character recognization over it.

In [13]:
import sys
sys.path.append("..")
from zAI import zImage

In [14]:
def cmd_OCR(bot, update):
    zi = zImage('newPhoto.png')
    ocr = zi.ocr(backend='Google')
    bot.send_message(chat_id = update.message.chat_id, text=ocr.text)

In [15]:
OCR_handler = CommandHandler('OCR', cmd_OCR)
dispatcher.add_handler(OCR_handler)

As it name suggest, the command /findFaces find the faces in the photo, crops they and sends they back one by one.

In [16]:
def cmd_findFaces(bot, update):
    zi = zImage('newPhoto.png')
    zi.find_faces(backend='Google')
    for i in range(len(zi.faces)):
        zface = zi.extract_face(n=i, margin=15)
        zface.save('newFace.png')
        bot.send_photo(chat_id = update.message.chat_id, photo=open('newFace.png','rb'))

In [17]:
findFaces_handler = CommandHandler('findFaces', cmd_findFaces)
dispatcher.add_handler(findFaces_handler)

The command /labels returns a list of the five labels that better describe our photo.

In [18]:
def cmd_labels(bot, update):
    zi = zImage('newPhoto.png')
    labels = zi.label(n=5, backend='Google')
    bot.send_message(chat_id = update.message.chat_id, text=', '.join(labels))

In [19]:
labels_handler = CommandHandler('labels', cmd_labels)
dispatcher.add_handler(labels_handler)

Finally, we initialice the bot to start working.

In [20]:
updater.start_polling()

In [21]:
updater.stop()

## Results

![alt text](figures/zAI-Telegram/Screenshot_01.png)
![alt text](figures/zAI-Telegram/Screenshot_02.png)
![alt text](figures/zAI-Telegram/Screenshot_03.png)
![alt text](figures/zAI-Telegram/Screenshot_04.png)
![alt text](figures/zAI-Telegram/Screenshot_05.png)
![alt text](figures/zAI-Telegram/Screenshot_06.png)